# 03 - Q&A Interaction Analysis

**Overview:** Analyze question-answer interactions, semantic similarity, question types, and topic patterns to surface conversational signals associated with evasion.

# 03 - Q&A Interaction Analysis

**Overview:** Analyze interactions between questions and answers to surface conversational patterns and question–answer dynamics.

In [ ]:
# Setup: topic modeling & embeddings
from datasets import load_dataset
import pandas as pd
from sentence_transformers import SentenceTransformer
from src.utils import set_seed

set_seed(42)

# Load dataset
print("Loading dataset FutureMa/EvasionBench...")
ds = load_dataset("FutureMa/EvasionBench")
if isinstance(ds, dict):
    ds = ds[list(ds.keys())[0]]
df = ds.to_pandas()
print("Dataset shape:", df.shape)

# Load lightweight sentence transformer for similarity
embedder = SentenceTransformer("all-MiniLM-L6-v2")
print("Embedder loaded: all-MiniLM-L6-v2")

# Example embedding
example = df.loc[0, "question"] + " " + df.loc[0, "answer"]
print("Sample text:\n", example[:200])

In [ ]:
# Semantic Similarity Analysis
print("=" * 60)
print("SEMANTIC SIMILARITY ANALYSIS")
print("=" * 60)

from sklearn.metrics.pairwise import cosine_similarity

# Encode questions and answers
print("\nEncoding questions and answers...")
question_embeddings = embedder.encode(df['question'].tolist(), show_progress_bar=True)
answer_embeddings = embedder.encode(df['answer'].tolist(), show_progress_bar=True)
print("Encoding complete.")

# Calculate cosine similarity
similarities = [cosine_similarity([q_emb], [a_emb])[0][0] 
                 for q_emb, a_emb in zip(question_embeddings, answer_embeddings)]

df['qa_similarity'] = similarities

# Statistics by label
print("\nSemantic similarity statistics by evasion category:")
sim_stats = df.groupby('eva4b_label').agg({
    'qa_similarity': ['mean', 'std', 'median', 'min', 'max']
}).round(3)
print(sim_stats)

# HYPOTHESIS: Evasive answers have lower semantic similarity to questions
print("\n📊 HYPOTHESIS: 'Evasive answers have lower semantic similarity to questions'")
direct_sim = df[df['eva4b_label'] == 'direct']['qa_similarity'].mean()
evasive_sim = df[df['eva4b_label'] == 'fully_evasive']['qa_similarity'].mean()
print(f"  Direct answers mean similarity: {direct_sim:.4f}")
print(f"  Evasive answers mean similarity: {evasive_sim:.4f}")
print(f"  Difference: {direct_sim - evasive_sim:.4f}")

# Mann-Whitney U test
stat, p_value = stats.mannwhitneyu(
    df[df['eva4b_label'] == 'direct']['qa_similarity'],
    df[df['eva4b_label'] == 'fully_evasive']['qa_similarity'],
    alternative='two-sided'
)
print(f"  Mann-Whitney U test p-value: {p_value:.2e}")
print(f"  Result: {'Significant difference (p < 0.05)' if p_value < 0.05 else 'No significant difference'}")

if p_value < 0.05:
    if evasive_sim < direct_sim:
        print("\n✅ HYPOTHESIS SUPPORTED: Evasive answers have significantly lower semantic similarity")
    else:
        print("\n❌ HYPOTHESIS CONTRADICTED: Evasive answers have HIGHER semantic similarity")

In [ ]:
# Question Type Classification
print("\n" + "=" * 60)
print("QUESTION TYPE CLASSIFICATION")
print("=" * 60)

# Extract question types based on wh-words and patterns
question_patterns = {
    'what': r'\bwhat\b',
    'why': r'\bwhy\b',
    'how': r'\bhow\b',
    'when': r'\bwhen\b',
    'where': r'\bwhere\b',
    'can_you': r'\bcan you\b|could you\b',
    'will_you': r'\bwill you\b|are you going to\b',
    'do_you': r'\bdo you\b|are you\b',
    'is': r'\bis\b|are\b',
    'other': r'(?i)(?!.*(what|why|how|when|where|can you|will you|do you|is\b|are\b)'
}

# Classify each question
question_type = []
import re
for q in df['question']:
    matched = False
    for qtype, pattern in question_patterns.items():
        if re.search(pattern, q, re.IGNORECASE):
            question_type.append(qtype)
            matched = True
            break
    if not matched:
        question_type.append('other')

df['question_type'] = question_type

print("\nQuestion type distribution:")
qtype_counts = df['question_type'].value_counts().sort_values(ascending=False)
for qtype, count in qtype_counts.items():
    print(f"  {qtype:12s}: {count:,} ({count/len(df)*100:.1f}%)")

# Evasion rate by question type
print("\nEvasion rate by question type:")
evasion_by_qtype = df.groupby('question_type').agg({
    'eva4b_label': lambda x: (x == 'fully_evasive').sum(),
    'total': 'size'
}).reset_index()
evasion_by_qtype['evasion_rate'] = (evasion_by_qtype['eva4b_label'] / evasion_by_qtype['total'] * 100)
evasion_by_qtype = evasion_by_qtype.sort_values('evasion_rate', ascending=False)
print(evasion_by_qtype[['question_type', 'evasion_rate', 'total']].to_string(index=False))

# Types with highest evasion
highest_evasion = evasion_by_qtype.iloc[0]['question_type']
print(f"\n⚠️ Question type '{highest_evasion}' elicits the highest evasion rate")

In [ ]:
# Topic Modeling on Questions (LDA)
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.manifold import TSNE
import random

print("\n" + "=" * 60)
print("TOPIC MODELING - QUESTIONS (LDA)")
print("=" * 60)

# Create documents for LDA
questions = df['question'].tolist()

# Vectorize
vectorizer = CountVectorizer(max_features=5000, stop_words='english', min_df=2)
doc_term_matrix = vectorizer.fit_transform(questions)
feature_names = vectorizer.get_feature_names_out()

# Train LDA model (try different number of topics)
n_topics = 15
lda = LatentDirichletAllocation(n_components=n_topics, random_state=42, max_iter=10)
lda.fit(doc_term_matrix)

print(f"\nTrained LDA with {n_topics} topics on {len(questions)} questions.")

# Display top words for each topic
print("\nTop 10 words per topic:")
for topic_idx, topic in enumerate(lda.components_, 1):
    top_words_idx = topic.argsort()[-10:][::-1]
    top_words = [feature_names[i] for i in top_words_idx]
    print(f"\nTopic {topic_idx}: {', '.join(top_words)}")

# Assign topics to questions
question_topics = lda.transform(doc_term_matrix)
df['question_topic'] = question_topics.argmax(axis=1)

# Topic distribution by evasion category
print("\nTopic distribution by evasion category:")
for label in ['direct', 'intermediate', 'fully_evasive']:
    subset = df[df['eva4b_label'] == label]
    dist = subset['question_topic'].value_counts().sort_index()
    print(f"\n{label.upper()}:")
    for topic, count in dist.head(5).items():
        print(f"  Topic {topic}: {count:,} ({count/len(subset)*100:.1f}%)")

In [ ]:
# Topic Modeling on Answers & Visualization
print("\n" + "=" * 60)
print("TOPIC MODELING - ANSWERS (NMF)")
print("=" * 60)

# Use NMF for better topic coherence
from sklearn.decomposition import NMF

answers = df['answer'].tolist()
tfidf_vectorizer = TfidfVectorizer(max_features=5000, stop_words='english', min_df=2)
tfidf = tfidf_vectorizer.fit_transform(answers)

# Train NMF
n_topics_answer = 15
nmf = NMF(n_components=n_topics_answer, random_state=42, max_iter=200)
nmf.fit(tfidf)

feature_names = tfidf_vectorizer.get_feature_names_out()

print(f"\nTrained NMF with {n_topics_answer} topics on answers.")

# Display topics
print("\nTop words per topic (answers):")
for topic_idx, topic in enumerate(nmf.components_, 1):
    top_words_idx = topic.argsort()[-10:][::-1]
    top_words = [feature_names[i] for i in top_words_idx]
    print(f"\nTopic {topic_idx}: {', '.join(top_words)}")

# Add t-SNE for visualization (optional - can be slow)
print("\n⚠️ Note: t-SNE visualization is computationally expensive and commented out by default.")
print("Uncomment the following cell to generate 2D topic visualization.")

# # Create t-SNE embedding
# print("Creating t-SNE visualization (this may take several minutes)...")
# answer_topics = nmf.transform(tfidf)
# tsne = TSNE(n_components=2, random_state=42, perplexity=30)
# tsne_results = tsne.fit_transform(answer_topics.toarray())

# # Plot
# plt.figure(figsize=(12, 10))
# colors = {'direct': '#2ecc71', 'intermediate': '#f39c12', 'fully_evasive': '#e74c3c'}
# for label, color in colors.items():
#     subset = df[df['eva4b_label'] == label]
#     plt.scatter(tsne_results[subset.index, 0], tsne_results[subset.index, 1], 
#                c=color, label=label, alpha=0.6, s=10)
# plt.legend()
# plt.title('t-SNE of Answer Topics by Evasion Label', fontsize=14, fontweight='bold')
# plt.xlabel('t-SNE Dimension 1')
# plt.ylabel('t-SNE Dimension 2')
# plt.savefig('notebooks/figures/03_tsne_topics.png', dpi=150, bbox_inches='tight')
# plt.show()

In [ ]:
# Answer Strategy Taxonomy (Manual Annotation Sample)
print("\n" + "=" * 60)
print("SAMPLE ANSWER STRATEGY TAXONOMY (50 per category)")
print("=" * 60)

# Create samples for manual coding
def strategy_codebook():
    """Return strategy codebook for manual annotation."""
    return """ANSWER STRATEGY CODEBOOK
=====================

Answer Types: STRATEGY CODES
---
1. DIRECT: Question is directly answered with specific information
   Examples: "Our Q4 revenue was $1.2B.", "Yes, we expect 5% growth."
   
2. PARTIAL: Some information provided, but question not fully addressed
   Examples: "We saw growth in [specific areas]", "That depends on [conditions]."
   
3. DEFLECTION: Answer shifts to different topic or question
   Examples: "Let me talk about our strategic priorities.", "Important to focus on customer success."
   
4. REFRAMING: Question is paraphrased, answer addresses reformulated question
   Examples: "Regarding growth drivers, we see...", "On the topic of profitability..."
   
5. VAGUE: Non-specific answer without clear stance or figures
   Examples: "We remain committed to growth.", "We're generally optimistic."
   
6. HEDGED: Qualifying language reduces commitment
   Examples: "We expect roughly...", "Growth could be around...", "It's hard to say exactly."
   
7. REFUSAL: Question is directly avoided or refused
   Examples: "I can't comment on that.", "We don't provide guidance."
"""

print(strategy_codebook())

# Sample rows for each category
samples_for_annotation = []
for label in df['eva4b_label'].unique():
    subset = df[df['eva4b_label'] == label].sample(min(50, len(df[df['eva4b_label'] == label]), random_state=42)
    samples_for_annotation.append(subset)

annotation_sample = pd.concat(samples_for_annotation)

# Save annotation sample
annotation_path = 'data/annotations/answer_strategy_annotation_sample.csv'
annotation_sample[['uid', 'question', 'answer', 'eva4b_label']].to_csv(annotation_path, index=False)

print(f"\n✅ Saved {len(annotation_sample)} samples to {annotation_path}")
print("    - 100 samples (50 per evasion category)")
print("    - Add a 'strategy' column with codes from codebook above")

## Key Findings

### Semantic Similarity
- Question-answer similarity [calculated] for all pairs
- Mean similarity: [to be filled]
- Hypothesis result: [To be filled after execution]

### Question Types
- "Why" questions elicit [X% evasion rate]
- "How" and "what" questions [higher/lower] evasion rates
- Suggests certain question types are prone to evasion

### Topic Modeling
- [X] topics identified from questions (LDA)
- [Y] topics identified from answers (NMF)
- Topics [correlated/didnt correlate] with evasion categories
- [Specific topics] tend to receive more evasive answers

### Answer Strategies
- Sample [50 per category] created for manual annotation
- Codebook with 7 strategy types provided
- Strategies to identify: DIRECT, PARTIAL, DEFLECTION, REFRAMING, VAGUE, HEDGED, REFUSAL

### Insights
- Evasive answers may [shift away from question topics]
- Question type is [modest/strong] predictor of evasion rate
- Semantic similarity [may/may not] be a reliable indicator

### Next Steps
- Manually annotate answer strategy samples
- Use identified patterns for feature engineering (Notebook 4)
- Consider question type as input feature for models

# 03 - Q&A Interaction Analysis

**Objectives:**
- Topic modeling on questions and answers
- Semantic similarity and question-type analysis
- Build answer strategy taxonomy